In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

from bs4 import BeautifulSoup # class for web scraping. It allows you to interact with HTML in a similar way to how you would interact with a web page using developer tools.
import requests

In [9]:
browser = webdriver.Edge("msedgedriver.exe")
browser.maximize_window()
browser.get("https://www.imdb.com/")

# access the web and accept cookies automatically: //*[@id="__next"]/div/div/div[2]/div/button[2]
try:
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div/div/div[2]/div/button[2]'))
    ).click()
    # Clicar en desplegable //*[@id="imdbHeader-navDrawerOpen"]/span
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]/span'))
    ).click()
    
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader"]/div[2]/aside[1]/div/div[2]/div/div[1]/span/div/div/ul/a[2]/span'))
    ).click()
    # sort by duration
    browser.find_element(By.XPATH, '//*[@id="sort-by-selector"]/option[8]').click()
except TimeoutException:
    pass

element = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/main/div/div[3]/section/div/div[2]/div/ul'))
)

table_html = element.get_attribute('outerHTML')


soup = BeautifulSoup(table_html, "html.parser")
titles = [title.text for title in soup.find_all('h3', class_='ipc-title__text')]
# Remove numbers from titles
titles = [title.split('.')[1] for title in titles]
titles = [title[1:] for title in titles]

califications = [calification.text for calification in soup.find_all('span', class_='ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating')]
# from califications  only whats left of "\xa0"
califications = [calification.split('\xa0')[0] for calification in califications]

years = [year.text for year in soup.find_all('span', class_='sc-b0691f29-8 ilsLEX cli-title-metadata-item')]
# from years only interested in what have 4 digits
years = [year for year in years if len(year) == 4]

durations = [duration.text for duration in soup.find_all('span', class_="sc-b0691f29-8 ilsLEX cli-title-metadata-item")]
# from durations only interested if contains "m"
durations = [duration for duration in durations if 'm' or 'h' in duration]

ratings = [rating.text for rating in soup.find_all('span', class_="sc-b0691f29-8 ilsLEX cli-title-metadata-item")]
ratings = [rating for rating in ratings if len(rating) <=2]
ratings = [rating for rating in ratings if len(rating) <=2]

In [ ]:
durations = [duration for duration in durations if 'm' or 'h' in duration]
dura

In [17]:
df['Rating'].value_counts()

Rating
A     49
18    25
13    20
12     8
7      7
16     5
2h     4
T      2
14     2
Name: count, dtype: int64

In [13]:
# Create a DataFrame with Title, Year, Duration and Calification
df = pd.DataFrame(list(zip(titles, years, durations, califications, ratings)), columns=['Title', 'Year', 'Duration', 'Calification', 'Rating'])
print(df)

                           Title  Year Duration Calification Rating
0     El moderno Sherlock Holmes  1924      45m          8,2      A
1                       El chico  1921    1h 8m          8,2      A
2    El maquinista de La General  1926   1h 18m          8,1      A
3            Antes del atardecer  2004   1h 20m          8,1     13
4                      Toy Story  1995   1h 21m          8,3      A
..                           ...   ...      ...          ...    ...
117               El apartamento  1960    2h 6m          8,3      A
118             El gran dictador  1940    2h 6m          8,4      A
119    El indomable Will Hunting  1997    2h 7m          8,3     16
120                    Cafarnaúm  2018    2h 7m          8,4     18
121    El tesoro de Sierra Madre  1948    2h 7m          8,2     13

[122 rows x 5 columns]


In [24]:
df.to_csv('../Outputs/top_250_by_duration.csv', index=False)

In [18]:
# Pinchar otra vez en menú
WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]/span'))
    ).click()
# Pinchar en Calendario de lanzamientos
WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader"]/div[2]/aside[1]/div/div[2]/div/div[1]/span/div/div/ul/a[1]/span'))
    ).click()
# Tabla de lanzamientos
element = WebDriverWait(browser, 10).until(
    EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/div[3]/section/section'))
)


table_html = element.get_attribute('outerHTML')
soup = BeautifulSoup(table_html, "html.parser")

# Buscar cada tabla: //*[@id="__next"]/main/div/div[3]/section/section/article[1]

# create a dataframe with movie and release date
df2 = pd.DataFrame(columns=['Date', 'Title', 'Genre'])

tables = soup.find_all('article')
for table in tables:
    date = table.find('h3', class_='ipc-title__text').text
    titles = [title.text for title in table.find_all('a', class_='ipc-metadata-list-summary-item__t')]
    genre = [genre.text for genre in table.find_all('span', class_='ipc-metadata-list-summary-item__li')[0]]
    # Add to dataframe
    for title in titles:
        df2.loc[len(df2)] = [date, title, genre]

In [19]:
df2.head()

,Date,Title,Genre
0,05 abr 2024,La primera profecía (2024),[Terror]
1,05 abr 2024,Pequeñas cartas indiscretas (2023),[Terror]
2,05 abr 2024,Freelance (2023),[Terror]
3,05 abr 2024,El consentimento (2023),[Terror]
4,05 abr 2024,El castillo a través del espejo (2022),[Terror]


In [25]:
df2.to_csv('../Outputs/new_movies.csv', index=False)

In [20]:
# Pinchar otra vez en menú
WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader-navDrawerOpen"]/span'))
    ).click()
# Pinchar en Calendario de lanzamientos
WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="imdbHeader"]/div[2]/aside[1]/div/div[2]/div/div[3]/span/div/div/ul/a[1]/span'))
    ).click()

WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/main/div/div[3]/div[2]/div/div[2]/ul/a[2]/span '))
    ).click()
WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="sidebar"]/div[1]/div/div[2]/div[16]/span[6]/a'))
    ).click()

In [21]:
df_awards = pd.DataFrame(columns=['Year', 'Genres'])
for i in range(2005,2025):
    my_url = browser.current_url
    my_url = my_url[0:37]+str(i)+my_url[41:]
    print(my_url)
    # WebDriverWait(browser, 10).until(
    #     EC.element_to_be_clickable((By.XPATH, '//*[@id="center-3-react"]/div/div/div[1]/h3/div[1]/div[2]/div[1]/div[1]/div[2]/div[2]/div[1]/span/span/a'))
    # ).click()
    # TAbla best picture
    element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="center-3-react"]/div/div/div[1]/h3/div[1]'))
    )
    table_html = element.get_attribute('outerHTML')
    soup = BeautifulSoup(table_html, "html.parser")
    refs = []
    for a in soup.find_all('a', href=True):
        refs.append(a['href'])
    refs = list(set( [ref for ref in refs if 'title' in ref]))
    genres_year = []
    for ref in refs:
        new_url = my_url[:21]+ref
        browser.get(new_url)
        tab = WebDriverWait(browser, 100).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/div[1]/div[2]'))
        )
        tab_html = tab.get_attribute('outerHTML')
        soup = BeautifulSoup(tab_html, "html.parser")
        genres = [genre.text for genre in soup.find_all('span', class_='ipc-chip__text')]
        for g in genres:
            genres_year.append(g)
    browser.get(my_url)

    df_awards.loc[len(df_awards)] = [i, genres_year]

    

    

https://www.imdb.com/event/ev0000003/2005/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2006/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2007/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2008/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2009/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2010/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2011/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2012/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2013/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2014/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2015/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2016/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2017/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2018/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2019/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2020/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2021/1/?ref_=ev_eh
https://www.imdb.com/event/ev0000003/2022/1/?ref

In [23]:
df_awards.tail()

,Year,Genres
15,2020,"[Drama, Biografía, Comedia, Drama, Biografía, ..."
16,2021,"[Acción, Biografía, Drama, Biografía, Crimen, ..."
17,2022,"[Drama, Misterio, Drama, Historia, Suspense, B..."
18,2023,"[Crimen, Drama, Musical, Comedia, Drama, Cienc..."
19,2024,"[Biografía, Drama, Música, Acción, Drama, Dram..."


In [26]:
df_awards.to_csv('../Outputs/awards.csv', index=False)